In [1]:
import requests
import pandas as pd
cfb_api_token = "9O2UGjD04RH9HNhr88stlq8c3PzEt3kio4D2N0P1DfHKwh705MNzpF7D9ZaGZpOA"
headers = { 'Authorization': f'Bearer {cfb_api_token}'}
NAME = "Name"
CFB_OVER = "CFB Over"
CFB_UNDER = "CFB Under"
CFB_FAV = "CFB Favorite"
CFB_DOG = "CFB Underdog"
NFL_OVER = "NFL Over"
NFL_UNDER = "NFL Under"
NFL_FAV = "NFL Favorite"
NFL_DOG = "NFL Underdog"
POINTS = "POINTS"
results = []

def make_request(url):
    res = requests.get(url, headers=headers)
    return res.json()

DK = "DraftKings"
ESPN = "ESPN Bet"
def retrieve_odds(odds):
    dk_odds = [x for x in odds if x['provider'] == DK]
    if len(dk_odds) > 0:
        return dk_odds[0]
    
    espn_odds = [x for x in odds if x['provider'] == ESPN]
    if len(espn_odds) > 0:
        return espn_odds[0]
    
    return None

def fatten_odds(data):
    results = []
    for item in data:
        if len(item['lines']) > 1:
            odds = retrieve_odds(item['lines'])
            item.update(odds)
        del item['lines']
        del item['seasonType']
        results.append(item)
    return results

College_results = make_request('https://api.collegefootballdata.com/lines?year=2024&week=1')
College_results = fatten_odds(College_results)
ncaa_df = pd.DataFrame(College_results)

picks = pd.read_csv('week-1.csv')
picks.columns.values[1] = NAME
picks.columns.values[2] = CFB_OVER
picks.columns.values[3] = CFB_UNDER
picks.columns.values[4] = CFB_FAV
picks.columns.values[5] = CFB_DOG


In [2]:
def split_string(str:str):
    split = str.split(" ")
    v_index = None
    odds_index = -1
    verse_array = ["v",'v.',"vs","vs."]
    
    for i in range(len(split)):
        word = split[i]
        if word.lower() in verse_array:
            v_index = i

        if any(char == "(" for char in word):
            odds_index = i
          
        if any((char.isdigit() or char == "(") for char in word):
            d_index = i
    away_team = " ".join(split[0:v_index])
    home_team = " ".join(split[v_index+1:odds_index])
    odds = " ".join(split[odds_index:])
    return {"awayTeam": away_team, "homeTeam":home_team, "odds": odds}


def retrieve_game(df:pd.DataFrame, game:str):
    obj = split_string(game)
    game1 = df.loc[(df["awayTeam"] == obj['awayTeam'])]
    if not game1.empty:
        return {"game": game1, "odds": obj["odds"],"home":obj["homeTeam"],"away":obj["awayTeam"], "found": True}
    game2 = df.loc[df["homeTeam"] == obj["homeTeam"]]

    if not game2.empty:
        return {"game": game2, "odds": obj["odds"],"home":obj["homeTeam"],"away":obj["awayTeam"], "found": True}
    
    game3 = df.loc[df["homeTeam"] == obj["awayTeam"]]
    if not game3.empty:
        return {"game": game3, "odds": obj["odds"],"home":obj["homeTeam"],"away":obj["awayTeam"], "found": True}
    
    game4 = df.loc[df["awayTeam"] == obj["homeTeam"]] 
    if not game4.empty:
        return {"game": game4, "odds": obj["odds"],"home":obj["homeTeam"],"away":obj["awayTeam"], "found": True}

    return {"found": False}
    
def return_values(df:pd.DataFrame,values):
    results = []
    for key in values:
        results.append(float(df.get(key).values[0]))
    return results


def calculate_under_results(df:pd.DataFrame, str:str):
    results = retrieve_game(df, str)
    game = results["game"]
    odds = results["odds"]
    has_odds = bool(game.get("provider").values[0])
    if has_odds:
        scores = return_values(game, ["overUnder", 'homeScore','awayScore'])
        return scores[0] >= scores[1] + scores[2]
    else:
        print(f"No odds for this game {str}")
        print(f"User odds {odds}")
        return False

def calculate_over_results(df:pd.DataFrame, str:str):
    results = retrieve_game(df, str)
    game = results["game"]
    odds = results["odds"]
    has_odds = bool(game.get("provider").values[0])
    if has_odds:
        scores = return_values(game, ["overUnder", 'homeScore','awayScore'])
        return scores[0] <= scores[1] + scores[2]
    else:
        print(f"No odds for this game {str}")
        print(f"User odds {odds}")
        return False


def calculate_favorite_results(df:pd.DataFrame, str:str):
    results = retrieve_game(df, str)
    if not results['found']:
        print(f"Not able to verify game: {str}")
    game = results["game"]
    odds = results["odds"]
    home = results["home"]
    away = results['away']
    scores = return_values(game, ['homeScore','awayScore'])
    number = odds[1:-1].split(" ")[-1]
    handicap = float(number)
    if home in odds:
        return  scores[1] <= scores[0] + handicap
    if away in odds:
        return  scores[0] <= scores[1] + handicap
    


def calculate_underdawg_results(df:pd.DataFrame, str:str):
    results = retrieve_game(df, str)
    if not results['found']:
        print(f"Not able to verify game: {str}")
    game = results["game"]
    odds = results["odds"]
    home = results["home"]
    away = results['away']
    scores = return_values(game, ['homeScore','awayScore'])
    number = odds[1:-1].split(" ")[-1]
    handicap = float(number)

    if home in odds:
        return  scores[1] <= scores[0] + handicap

    if away in odds:
        return  scores[0] <= scores[1] + handicap
    


def calculate_points(row):
     points = 0
     ncaa_over = calculate_over_results(ncaa_df, row.get(CFB_OVER))
     ncaa_under = calculate_under_results(ncaa_df, row.get(CFB_UNDER))
     ncaa_fav = calculate_favorite_results(ncaa_df, row.get(CFB_FAV))
     ncaa_dog = calculate_underdawg_results(ncaa_df, row.get(CFB_DOG))

     if ncaa_over:
        points +=1 
     if ncaa_under:
         points +=1
     if ncaa_fav:
         points +=1
     if ncaa_dog:
         points +=1
         
     results.append([row.get(NAME),ncaa_over, ncaa_under, ncaa_fav, ncaa_dog, points ])
     return points

In [3]:
points = []
for index, row in picks.iterrows():
    points.append(calculate_points(row))

picks[POINTS] = points

In [4]:
picks

,Timestamp,Name,CFB Over,CFB Under,CFB Favorite,CFB Underdog,POINTS
0,8/28/2024 22:27:14,Matthew G,Clemson v. Georgia 48.5,Penn State v. West Virginia 51.5,TCU v. Stanford (TCU -9.5),Notre Dame v. Texas AM (Notre Dame +3),2
1,8/29/2024 13:59:18,Michael Rubino,Idaho vs Oregon 62.5,Fresno State vs Michigan 45.5,Western Kentucky vs Alabama (Alabama -31.5),Miami vs Florida (Florida +2.5),2
2,8/29/2024 17:04:04,Lito,Howard vs Rutgers 51,North Carolina V. Jacksonville State 54,Ohio State vs Akron (Ohio State -48.5),Larmar v. Texas State (Larmar +31),2
3,8/29/2024 18:55:05,Bri Rubino,Akron V. Ohio State 58,Western Kentucky V. Alabama 59.5,Penn State V. West Virgin (Penn State -8),Boston College V. Florida State (Boston Colleg...,2
4,8/29/2024 19:13:34,Max,Tennessee v. Chattanooga 56.5,Penn State v. West Virginia 51.5,Texas A&M v. Notre Dame (Texas A&M -3),Elon +25.5 v. Duke (Elon +25.5),3
5,8/29/2024 20:57:17,Madison Rubino,Austin Peay V. Louisville 52.5,TCU V. Stanford 59.5,Eastern Michigan V. UMass (UMass -2.5),Indiana State V. Purdue (Indiana State +33.5),1
6,8/30/2024 12:33:30,Nick Delacruz,Michigan V. Fresno State 45.5,Oregon V. Idaho 62.5,Georgia V. Clemson (Georgia -12.5),Notre Dame v. Texas AM (Notre Dame +3),1
7,8/30/2024 13:14:19,Mama Golden,Miami V Northwestern 39.5,Akron V. Ohio State University 57,Syracuse V Ohio (Syracuse -17.5),Clemson V Georgia (Celson +11.5),2
8,8/30/2024 14:29:40,Joe Rubino,Oklahoma V Temple 57.5,Clemson v. Georgia 48.5,Oklahoma v. Temple (Oklahoma -43),Colorado State v. Texas (Colorado State +34),1
9,8/30/2024 16:43:01,Drew,Michigan State vs. FAU 45.5,Virginia Tech Vs Vanderbilt 48.5,Ohio State vs Akron (Ohio -48.5),Miami (OH) vs Northwestern (Miami (OH) +3.5),0


In [7]:
results_df = pd.DataFrame(results)



,0,1,2,3,4,5
0,Matthew G,False,True,False,True,2
1,Michael Rubino,False,True,True,False,2
2,Lito,False,True,False,True,2
3,Bri Rubino,False,False,True,True,2
4,Max,False,True,True,True,3
5,Madison Rubino,False,True,False,False,1
6,Nick Delacruz,False,False,False,True,1
7,Mama Golden,True,True,False,None,2
8,Joe Rubino,False,True,False,False,1
9,Drew,False,False,None,None,0
